In [ ]:
import os
from pyspark.sql import SparkSession

# Configurations

In [ ]:
dbConnectionUrl = "jdbc:mysql://mysql:3306/sakila"
dbProperties = {
    "driver":   "com.mysql.jdbc.Driver",
    "user":     "root",
    "password": "SuperSecr3t"
}

In [ ]:
minioConfigs = {
    "spark.hadoop.fs.s3a.impl":                "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "spark.hadoop.fs.s3a.endpoint":            "http://minio:9000",
    "spark.hadoop.fs.s3a.access.key":          "admin",
    "spark.hadoop.fs.s3a.secret.key":          "SuperSecr3t",
    "spark.hadoop.fs.s3a.path.style.access":   True
}

In [ ]:
from pyspark import SparkConf

conf = SparkConf()
conf.setAll(minioConfigs.items())
conf.set("spark.jars.packages", "mysql:mysql-connector-java:8.0.18")

In [ ]:
spark = SparkSession.builder\
        .appName("Teko::Summit")\
        .config(conf=conf)\
        .getOrCreate()

# Read from MySQL

In [ ]:
df = spark.read.jdbc(dbConnectionUrl, "actor", properties=dbProperties)

In [ ]:
df.show(5)

# Write to MinIO

In [ ]:
df.write.parquet("s3a://teko-summit/actor", mode="overwrite")

In [ ]:
df_minio = spark.read.parquet("s3a://teko-summit/actor")

In [ ]:
df_minio.show(100)